In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
df = pd.read_csv(r'pjs_train.csv', nrows = 5000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     3644 non-null   int64 
 1   ticket_number  3644 non-null   object
 2   response       3644 non-null   object
 3   nps            3644 non-null   object
 4   csat           3505 non-null   object
 5   Term           3359 non-null   object
 6   Tier_2         3359 non-null   object
 7   Tier_1         3359 non-null   object
 8   entities       3644 non-null   object
 9   entity_type    3644 non-null   object
 10  entity_start   3644 non-null   int64 
 11  entity_end     3644 non-null   int64 
 12  num            3644 non-null   int64 
dtypes: int64(4), object(9)
memory usage: 370.2+ KB


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import datasets
from datasets import Dataset, DatasetDict

prob_df = df['response'].to_frame()

ds = Dataset.from_pandas(prob_df)

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [ ]:
# Datasets 2.11.0
from datasets import load_dataset
#dataset = load_dataset("argilla/gutenberg_spacy-ner", split="train")
#Transformers 4.27.4, Torch 2.0.0+cu118,
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    pipeline
)
from transformers.pipelines.pt_utils import KeyDataset

# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

pipe = pipeline('text-classification', model="j-hartmann/emotion-english-distilroberta-base", device = device)
results = pipe(KeyDataset(ds, "response"))

# original = []
# temp_ent = []
# entities = []

# for idx, extracted_entities in enumerate(results):
#     temp_ent = []
#     print('idx: ', idx)
#     print("Original text:\n{}".format(ds[idx]["response"]))
#     print("Extracted entities:")
#     #original.append(dataset[idx]["text"])
#     for entity in extracted_entities:
#         temp_ent.append(entity)
#         print(entity)
#     entities.append(temp_ent)

In [ ]:
emotion_label = []

for idx, extracted_entities in enumerate(results):
    emotion_label.append(extracted_entities['label'])

df['emotion'] = emotion_label

In [ ]:
df.to_csv('emotion_append.csv')